In [1]:
!pip3 install kaggle  >> /dev/null

!mkdir -p ~/.kaggle

!echo '{"username":"abhignasree","key":"c2b31497de3865bbd63f5007905aff51"}' > ~/.kaggle/kaggle.json

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip >> /dev/null

 99% 2.28G/2.29G [00:25<00:00, 147MB/s]
100% 2.29G/2.29G [00:25<00:00, 98.2MB/s]


In [2]:
!pip install keras.applications.resnet50

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement keras.applications.resnet50 (from versions: none)
ERROR: No matching distribution found for keras.applications.resnet50


In [4]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.models import Model
#from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input

from keras.preprocessing.image import ImageDataGenerator

# Image size
IMAGE_SIZE = [224, 224]

# Paths to train and test directories
train_path = '/content/chest_xray/train'
valid_path = '/content/chest_xray/test'

# Load the ResNet50 model
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Global average pooling layer
x = GlobalAveragePooling2D()(resnet.output)

# Output layer
prediction = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=10, class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path, target_size=(224, 224), batch_size=10, class_mode='categorical')

# Train the model
history = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)





94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [5]:
# Save the model
model.save('chest_xray_resnet.h5')

# Load the model
model = keras.models.load_model('chest_xray_resnet.h5')

In [7]:
import numpy as np
# Load and preprocess the image
img_path = '/content/chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg'
img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)

# Make predictions
classes = model.predict(img_data)
result = int(classes[0][0])

# Print the result
if result == 0:
    print("Person is affected by PNEUMONIA")
else:
    print("Result is normal")


1/1 [==============================] - 0s 332ms/step
Result is normal


In [9]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# Define the AlexNet model architecture
input_layer = Input(shape=IMAGE_SIZE + [3])
x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = Conv2D(256, (5, 5), activation='relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = Conv2D(384, (3, 3), activation='relu')(x)
x = Conv2D(384, (3, 3), activation='relu')(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=10, class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path, target_size=(224, 224), batch_size=10, class_mode='categorical')

# Train the model
history = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 384)         8851

In [11]:
# Save the model
model.save('chest_xray_alexnet.h5')

# Load the model
model = keras.models.load_model('chest_xray_alexnet.h5')

In [13]:
# Load and preprocess the image
img_path = '/content/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg'
img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)

# Make predictions
classes = model.predict(img_data)
result = int(classes[0][0])

# Print the result
if result == 0:
    print("Person is affected by PNEUMONIA")
else:
    print("Result is normal")

1/1 [==============================] - 0s 49ms/step
Person is affected by PNEUMONIA


In [19]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os


In [20]:
# Set the image size
IMAGE_SIZE = (224, 224)


# Calculate the number of classes based on the subdirectories in the training directory
num_classes = len(os.listdir(train_path))

# Create the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512, activation='relu')(x)

# Add an output layer with the number of classes
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [21]:
# Create data generators for training and validation data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=IMAGE_SIZE, batch_size=10, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=IMAGE_SIZE, batch_size=10, class_mode='categorical')

# Train the model
model.fit_generator(train_generator, validation_data=valid_generator, epochs=1, steps_per_epoch=len(train_generator), validation_steps=len(valid_generator))


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
522/522 [==============================] - 742s 1s/step - loss: 0.3111 - accuracy: 0.8589 - val_loss: 0.3600 - val_accuracy: 0.8413


In [22]:
# Save the model
model.save('inception_model.h5')

In [23]:
# Load and preprocess the image
img_path = '/content/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg'
img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)

# Make predictions
classes = model.predict(img_data)
result = int(classes[0][0])

# Print the result
if result == 0:
    print("Person is affected by PNEUMONIA")
else:
    print("Result is normal")

1/1 [==============================] - 2s 2s/step
Person is affected by PNEUMONIA
